In [1]:
# imports/setup

import s3fs
import boto3
import json
import pandas as pd
from IPython.display import display

s3 = boto3.client('s3')

In [2]:
# read data

df_pr = pd.read_csv('s3://mjw-cloudquest-bls-data/pr_data_0_Current', delimiter='\t')
df_pr = df_pr.rename(columns=lambda x: x.strip()) # whitespace in column names!!

pop_raw = s3.get_object(Bucket='mjw-cloudquest-bls-data', Key='pop_data/2024-05-29T00-35-33Z/data.json')
pop_data = json.loads(pop_raw['Body'].read().decode('utf-8'))
df_pop = pd.DataFrame(pop_data['data'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [3]:
# Using the dataframe from the population data API (Part 2), generate the mean 
# and the standard deviation of the annual US population across the years [2013, 2018] inclusive.
df_pop_filtered = df_pop[df_pop['ID Year'] < 2019]
print(f"The mean of the populations from 2013 to 2018 is {df_pop_filtered['Population'].mean()}")
print(f"The standard deviation of the populations from 2013 to 2018 is {df_pop_filtered['Population'].std()}")

The mean of the populations from 2013 to 2018 is 317437383.0
The standard deviation of the populations from 2013 to 2018 is 4257089.5415293295


In [4]:
#Using the dataframe from the time-series (Part 1), 
#For every series_id, find the best year: the year with the max/largest sum of "value" 
#for all quarters in that year. Generate a report with each series id, the best year for that series, 
#and the summed value for that year.
df_pr
grouped = df_pr.groupby(['series_id', 'year', 'period'])['value'].sum().reset_index()

# Group by series id and year again, and find the year with the maximum sum for each series
best_years = grouped.groupby(['series_id', 'year'])['value'].sum().reset_index()
best_years = best_years.loc[best_years.groupby('series_id')['value'].idxmax()]

best_years[['series_id', 'year', 'value']]

,series_id,year,value
27,PRS30006011,2022,20.500
57,PRS30006012,2022,17.100
63,PRS30006013,1998,704.125
105,PRS30006021,2010,17.600
135,PRS30006022,2010,12.500
...,...,...,...
8130,PRS88003192,2002,282.800
8180,PRS88003193,2023,766.310
8208,PRS88003201,2022,36.100
8237,PRS88003202,2022,28.900
